In [1]:
import time

In [33]:
"""
    N-body simulation.
"""

PI = 3.14159265358979323
SOLAR_MASS = 4 * PI * PI
DAYS_PER_YEAR = 365.24

BODIES = {
    'sun': ([0.0, 0.0, 0.0], [0.0, 0.0, 0.0], SOLAR_MASS),

    'jupiter': ([4.84143144246472090e+00,
                 -1.16032004402742839e+00,
                 -1.03622044471123109e-01],
                [1.66007664274403694e-03 * DAYS_PER_YEAR,
                 7.69901118419740425e-03 * DAYS_PER_YEAR,
                 -6.90460016972063023e-05 * DAYS_PER_YEAR],
                9.54791938424326609e-04 * SOLAR_MASS),

    'saturn': ([8.34336671824457987e+00,
                4.12479856412430479e+00,
                -4.03523417114321381e-01],
               [-2.76742510726862411e-03 * DAYS_PER_YEAR,
                4.99852801234917238e-03 * DAYS_PER_YEAR,
                2.30417297573763929e-05 * DAYS_PER_YEAR],
               2.85885980666130812e-04 * SOLAR_MASS),

    'uranus': ([1.28943695621391310e+01,
                -1.51111514016986312e+01,
                -2.23307578892655734e-01],
               [2.96460137564761618e-03 * DAYS_PER_YEAR,
                2.37847173959480950e-03 * DAYS_PER_YEAR,
                -2.96589568540237556e-05 * DAYS_PER_YEAR],
               4.36624404335156298e-05 * SOLAR_MASS),

    'neptune': ([1.53796971148509165e+01,
                 -2.59193146099879641e+01,
                 1.79258772950371181e-01],
                [2.68067772490389322e-03 * DAYS_PER_YEAR,
                 1.62824170038242295e-03 * DAYS_PER_YEAR,
                 -9.51592254519715870e-05 * DAYS_PER_YEAR],
                5.15138902046611451e-05 * SOLAR_MASS)}

def advance(dt):
    '''
        advance the system one timestep
    '''
    bodies = BODIES
    seenit = set()
    for body1 in bodies.keys():
        for body2 in bodies.keys():
            if (body1 != body2) and not (body2 in seenit):
                ([x1, y1, z1], v1, m1) = bodies[body1]
                ([x2, y2, z2], v2, m2) = bodies[body2]
                (dx, dy, dz) = (x1-x2, y1-y2, z1-z2)
                
                mag = dt * ((dx * dx + dy * dy + dz * dz) ** (-1.5))
                v1_b = m2 * mag
                v1[0] -= dx * v1_b
                v1[1] -= dy * v1_b
                v1[2] -= dz * v1_b

                v2_b = m1 * mag
                v2[0] += dx * v2_b
                v2[1] += dy * v2_b
                v2[2] += dz * v2_b
                seenit.add(body1)
        
    for body in bodies.keys():
        (r, [vx, vy, vz], m) = bodies[body]
        r[0] += dt * vx
        r[1] += dt * vy
        r[2] += dt * vz
    
def report_energy(e=0.0):
    '''
        compute the energy and return it so that it can be printed
    '''
    seenit = set()
    bodies = BODIES
    
    for body1 in bodies.keys():
        for body2 in bodies.keys():
            if (body1 != body2) and not (body2 in seenit):
                ((x1, y1, z1), v1, m1) = bodies[body1]
                ((x2, y2, z2), v2, m2) = bodies[body2]
                (dx, dy, dz) = (x1-x2, y1-y2, z1-z2)
                e -= (m1 * m2) / ((dx * dx + dy * dy + dz * dz) ** 0.5)
                seenit.add(body1) 
        
    for body in bodies.keys():
        (r, [vx, vy, vz], m) = bodies[body]
        e += m * (vx * vx + vy * vy + vz * vz) / 2.
        
    return e

def offset_momentum(ref, px=0.0, py=0.0, pz=0.0):
    '''
        ref is the body in the center of the system
        offset values from this reference
    '''
    bodies = BODIES
    for body in bodies.keys():
        (r, [vx, vy, vz], m) = bodies[body]
        px -= vx * m
        py -= vy * m
        pz -= vz * m
        
    (r, v, m) = ref
    v[0] = px / m
    v[1] = py / m
    v[2] = pz / m


def nbody(loops, reference, iterations):
    '''
        nbody simulation
        loops - number of loops to run
        reference - body at center of system
        iterations - number of timesteps to advance
    '''
    # Set up global state
    offset_momentum(BODIES[reference])
    
    # local variables for function calls
    advance_001 = advance
    print_energy = report_energy
    
    for _ in range(loops):
        start = time.time() #add
        
        for _ in range(iterations):
            advance_001(0.01)
        print(print_energy())
        
        end = time.time() #add
        print("time: ", end - start) #add

if __name__ == '__main__':
    nbody(100, 'sun', 20000)

-0.16908926275527025
time:  0.2861919403076172
-0.1690524049239042
time:  0.2867121696472168
-0.169014744045289
time:  0.29660868644714355
-0.16902307738079617
time:  0.29264044761657715
-0.16907985939165196
time:  0.2971034049987793
-0.16908378513401653
time:  0.2909669876098633
-0.16904612370292355
time:  0.31049489974975586
-0.16901416322874255
time:  0.2982814311981201
-0.16902778605966304
time:  0.30752110481262207
-0.1690837125696274
time:  0.3020622730255127
-0.1690807806762558
time:  0.31148815155029297
-0.1690447415801465
time:  0.30355191230773926


KeyboardInterrupt: 

In [16]:
%timeit nbody(100, 'sun', 1000)

1.56 s ± 11.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [18]:
%timeit nbody2(100, 'sun', 1000)

1.5 s ± 19.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [32]:
X = 10
def test():
    for _ in range(100):
        X += X
test()
print(X)
%timeit -n 100 -r 3 test()

UnboundLocalError: local variable 'X' referenced before assignment

In [25]:
X = 10
def test2():
    x = X
    for _ in range(100):
        x += x
    return x
x = test2()
print(x)
%timeit -n 100 -r 3 test2()

12676506002282294014967032053760
3.74 µs ± 11.5 ns per loop (mean ± std. dev. of 3 runs, 100 loops each)
